<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 扩展 Tiktoken BPE 分词器以添加新词汇


- 本笔记讲解了如何扩展现有的 BPE 分词器；具体来说，我们将重点介绍如何针对流行的 [tiktoken](https://github.com/openai/tiktoken) 实现进行扩展。
- 有关分词的一般介绍，请参考 [第2章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb) 以及 BPE 从零开始教程 [链接]。
- 例如，假设我们有一个 GPT-2 分词器，并希望对以下文本进行编码：


In [1]:
import tiktoken

base_tokenizer = tiktoken.get_encoding("gpt2")
sample_text = "Hello, MyNewToken_1 is a new token. <|endoftext|>"

token_ids = base_tokenizer.encode(sample_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[15496, 11, 2011, 3791, 30642, 62, 16, 318, 257, 649, 11241, 13, 220, 50256]


- 对每个 token ID 进行迭代，可以帮助我们更好地理解这些 token ID 是如何通过词表进行解码的：


In [2]:
for token_id in token_ids:
    print(f"{token_id} -> {base_tokenizer.decode([token_id])}")

15496 -> Hello
11 -> ,
2011 ->  My
3791 -> New
30642 -> Token
62 -> _
16 -> 1
318 ->  is
257 ->  a
649 ->  new
11241 ->  token
13 -> .
220 ->  
50256 -> <|endoftext|>


- 如上所示， `"MyNewToken_1"` 被拆分成了 5 个单独的子词 token —— 对于 BPE 在处理未知词时，这是正常行为。  
- 然而，假设它是一个特殊 token，我们希望将其编码为一个单独的 token，就像其他某些单词或 `"<|endoftext|>"` 一样；本笔记本将解释如何实现这一点。


&nbsp;
## 1. 添加特殊 token


- 注意，我们必须将新 token 添加为特殊 token；原因在于，这些新 token 在 tokenizer 训练过程中没有生成对应的 "merges" —— 即使有这些 "merges"，在不破坏现有分词方案的情况下将它们加入也非常困难（参考 BPE from scratch 教程 [link] 了解 "merges" 的概念）
- 假设我们想添加 2 个新 token：


In [3]:
# Define custom tokens and their token IDs
custom_tokens = ["MyNewToken_1", "MyNewToken_2"]
custom_token_ids = {
    token: base_tokenizer.n_vocab + i for i, token in enumerate(custom_tokens)
}

- 接下来，我们创建一个自定义的 `Encoding` 对象，用于保存我们的特殊 token，代码如下：


In [4]:
# Create a new Encoding object with extended tokens
extended_tokenizer = tiktoken.Encoding(
    name="gpt2_custom",
    pat_str=base_tokenizer._pat_str,
    mergeable_ranks=base_tokenizer._mergeable_ranks,
    special_tokens={**base_tokenizer._special_tokens, **custom_token_ids},
)

- 就这样，我们现在可以检查它是否可以对示例文本进行编码：


- 如我们所见，新的标记 `50257` 和 `50258` 现在已经在输出中被编码：


In [5]:
special_tokens_set = set(custom_tokens) | {"<|endoftext|>"}

token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)
print(token_ids)

[36674, 2420, 351, 220, 50257, 290, 220, 50258, 13, 220, 50256]


- 同样，我们也可以逐个标记查看编码情况：


In [6]:
for token_id in token_ids:
    print(f"{token_id} -> {extended_tokenizer.decode([token_id])}")

36674 -> Sample
2420 ->  text
351 ->  with
220 ->  
50257 -> MyNewToken_1
290 ->  and
220 ->  
50258 -> MyNewToken_2
13 -> .
220 ->  
50256 -> <|endoftext|>


- 如上所示，我们已经成功更新了分词器（tokenizer）。
- 但是，要将其用于预训练的大语言模型（LLM），我们还需要更新 LLM 的嵌入层（embedding layer）和输出层（output layer），这些将在下一节中讨论。


&nbsp;
## 2. 更新预训练的大语言模型（LLM）


- 在本节中，我们将介绍在更新分词器（tokenizer）后，如何更新现有的预训练大语言模型（LLM）。
- 为此，我们使用书中主章节里使用的原始预训练 GPT-2 模型。


### 2.1 加载一个预训练的 GPT 模型

In [7]:
from llms_from_scratch.ch05 import download_and_load_gpt2
# For llms_from_scratch installation instructions, see:
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|███████████████████████████| 77.0/77.0 [00:00<00:00, 34.4kiB/s]
encoder.json: 100%|███████████████████████| 1.04M/1.04M [00:00<00:00, 4.78MiB/s]
hparams.json: 100%|█████████████████████████| 90.0/90.0 [00:00<00:00, 24.7kiB/s]
model.ckpt.data-00000-of-00001: 100%|███████| 498M/498M [00:33<00:00, 14.7MiB/s]
model.ckpt.index: 100%|███████████████████| 5.21k/5.21k [00:00<00:00, 1.05MiB/s]
model.ckpt.meta: 100%|██████████████████████| 471k/471k [00:00<00:00, 2.33MiB/s]
vocab.bpe: 100%|████████████████████████████| 456k/456k [00:00<00:00, 2.45MiB/s]


In [8]:
from llms_from_scratch.ch04 import GPTModel
# For llms_from_scratch installation instructions, see:
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

# Define model configurations in a dictionary for compactness
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # Example model name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

### 2.2 使用预训练的 GPT 模型

- 接下来，我们来看下面这段示例文本，并分别使用原始的分词器（tokenizer）和更新后的分词器对其进行分词：


In [9]:
sample_text = "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>"

original_token_ids = base_tokenizer.encode(
    sample_text, allowed_special={"<|endoftext|>"}
)

In [10]:
new_token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)

- 现在，我们将原始的 token ID 输入到 GPT 模型中进行推理：


In [11]:
import torch

with torch.no_grad():
    out = gpt(torch.tensor([original_token_ids]))

print(out)

tensor([[[ 0.2204,  0.8901,  1.0138,  ...,  0.2585, -0.9192, -0.2298],
         [ 0.6745, -0.0726,  0.8218,  ..., -0.1768, -0.4217,  0.0703],
         [-0.2009,  0.0814,  0.2417,  ...,  0.3166,  0.3629,  1.3400],
         ...,
         [ 0.1137, -0.1258,  2.0193,  ..., -0.0314, -0.4288, -0.1487],
         [-1.1983, -0.2050, -0.1337,  ..., -0.0849, -0.4863, -0.1076],
         [-1.0675, -0.5905,  0.2873,  ..., -0.0979, -0.8713,  0.8415]]])


- 如上所示，模型运行正常（此处为了简洁起见，代码仅展示了原始输出结果，并未将其转换回文本形式；若需了解如何将模型输出转换为文本，可参考第 5 章第 5.3.3 节中的 `generate` 函数 [链接]）。


- 那么，如果我们现在尝试将**更新后的分词器**生成的 token ID 输入到模型中，会发生什么呢？


```python
with torch.no_grad():
    gpt(torch.tensor([new_token_ids]))

print(out)

...
# IndexError: index out of range in self
```

- 如上所示，这会导致一个 **index error（索引错误）**。  
- 其原因在于：GPT 模型的输入嵌入层（input embedding layer）和输出层（output layer）都假设词汇表（vocabulary）的大小是固定的。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/extend-tiktoken/gpt-updates.webp" width="400px">


&nbsp;
### 2.3 更新嵌入层（Embedding Layer）


- 我们先从更新嵌入层（embedding layer）开始。
- 首先要注意，嵌入层当前有 **50,257** 个条目，这个数字正好对应于词汇表（vocabulary）的大小。


In [12]:
gpt.tok_emb

Embedding(50257, 768)

- 我们希望通过增加 **2** 个新条目来扩展嵌入层（embedding layer）。
- 简而言之，我们需要创建一个更大的嵌入层，然后将原有嵌入层中的权重值复制到新嵌入层中。


In [13]:
num_tokens, emb_size = gpt.tok_emb.weight.shape
new_num_tokens = num_tokens + 2

# Create a new embedding layer
new_embedding = torch.nn.Embedding(new_num_tokens, emb_size)

# Copy weights from the old embedding layer
new_embedding.weight.data[:num_tokens] = gpt.tok_emb.weight.data

# Replace the old embedding layer with the new one in the model
gpt.tok_emb = new_embedding

print(gpt.tok_emb)

Embedding(50259, 768)


- 如上所示，我们现在成功扩展了嵌入层（embedding layer）。


&nbsp;
### 2.4 更新输出层


- 接下来，我们需要扩展输出层，该层具有 50,257 个输出特征，对应于与嵌入层相同的词汇表大小（顺便提一下，你可能会觉得附加材料很有用，其中讨论了 PyTorch 中线性层与嵌入层的相似性）。


In [14]:
gpt.out_head

Linear(in_features=768, out_features=50257, bias=False)

- 扩展输出层的操作步骤与扩展嵌入层类似：


In [15]:
original_out_features, original_in_features = gpt.out_head.weight.shape

# Define the new number of output features (e.g., adding 2 new tokens)
new_out_features = original_out_features + 2

# Create a new linear layer with the extended output size
new_linear = torch.nn.Linear(original_in_features, new_out_features)

# Copy the weights and biases from the original linear layer
with torch.no_grad():
    new_linear.weight[:original_out_features] = gpt.out_head.weight
    if gpt.out_head.bias is not None:
        new_linear.bias[:original_out_features] = gpt.out_head.bias

# Replace the original linear layer with the new one
gpt.out_head = new_linear

print(gpt.out_head)

Linear(in_features=768, out_features=50259, bias=True)


- 让我们先在原始的 token ID 上尝试这个更新后的模型：


In [16]:
with torch.no_grad():
    output = gpt(torch.tensor([original_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [ 0.1200, -0.1027,  2.0549,  ..., -0.1519, -0.2096,  0.5651],
         [-1.1920, -0.1819, -0.0981,  ..., -0.1108,  0.8435, -0.3771],
         [-1.0612, -0.5674,  0.3229,  ...,  0.8383, -0.7121, -0.4850]]])


- 接下来，让我们在更新后的 token 上尝试：


In [17]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [-0.0656, -1.2451,  0.7957,  ..., -1.2124,  0.1044,  0.5088],
         [-1.1561, -0.7380, -0.0645,  ..., -0.4373,  1.1401, -0.3903],
         [-0.8961, -0.6437, -0.1667,  ...,  0.5663, -0.5862, -0.4020]]])


- 正如我们所看到的，模型可以在扩展后的 token 集上正常工作  
- 在实际操作中，我们希望现在对模型进行微调（或持续预训练）（特别是新的嵌入层和输出层），使用包含新 token 的数据


**关于权重绑定的说明**

- 如果模型使用权重绑定，即嵌入层和输出层共享相同的权重（类似于 Llama 3 [链接]），那么更新输出层会简单得多  
- 在这种情况下，我们只需将嵌入层的权重复制过来即可：


In [18]:
gpt.out_head.weight = gpt.tok_emb.weight

In [19]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))